# Modeling and interpretability

In [1]:
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix,accuracy_score,make_scorer,precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTENC, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import plotly.express as px

In [2]:

df = pd.read_csv("customer_booking.csv", encoding="ISO-8859-1").copy()

# Lead bins (create as 'lead_bin' to match cat_cols)
lead_bins = [-np.inf, 3, 7, 14, 30, 90, np.inf]
lead_bins_label = ['0-3','4-7','8-14','15-30','31-90','90+']
df['lead_bin'] = pd.cut(df['purchase_lead'].clip(lower=0),
                        bins=lead_bins, labels=lead_bins_label, include_lowest=True)

# Length of stay bins (already named 'stay_bin')
stay_bins = [-np.inf, 3, 7, 14, 30, 90, np.inf]
stay_bins_label = ['0-3','4-7','8-14','15-30','31-90','90+']
df['stay_bin'] = pd.cut(df['length_of_stay'].clip(lower=0),
                        bins=stay_bins, labels=stay_bins_label, include_lowest=True)

# Part of day (consistent label case helps later one-hot)
def day_part(h):
    if 0 <= h <= 5:   return 'Night'
    if 6 <= h <= 11:  return 'Morning'
    if 12 <= h <= 17: return 'Afternoon'
    return 'Evening'
df['daypart'] = df['flight_hour'].apply(day_part)

target = 'booking_complete'

cat_cols = [
    'lead_bin','stay_bin','daypart','sales_channel','trip_type','flight_day',
    'route','booking_origin','wants_extra_baggage','wants_preferred_seat','wants_in_flight_meals'
]
num_cols = ['purchase_lead','length_of_stay','flight_hour','flight_duration','num_passengers']


# Optional safety: strip spaces and check existence
df.columns = df.columns.str.strip()

missing = [c for c in cat_cols + num_cols if c not in df.columns]
if missing:
    raise KeyError(f"Missing columns: {missing}")

X=df.drop(columns=target)
y=df[target].astype(int)


## train/test(stratified)

In [3]:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)



##  encoding

In [4]:
preprocessor=ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num",QuantileTransformer(), num_cols)
    ]
)
X=df.drop(columns=[target])
y=df['booking_complete'].astype(int)




## Baseline: Logistic Regression

In [5]:
model=LogisticRegression(max_iter=1000,random_state=42,solver='saga')

pipe=Pipeline(
    [
        ("preprocessor", preprocessor),
          
        ("model", model)
    ]
)
mod=GridSearchCV(
    estimator=pipe,
    param_grid={
        'model__max_iter':[100,200,300,400,500,600,700,800,900,1000,2000],
        'model__solver':['saga','liblinear'],
        'model__class_weight':[{0:1,1:v}for v in np.linspace(1,20,10)]
    },
    cv=5,
    scoring='f1',
    n_jobs=2
)
mod.fit(X_test,y_test)
y_pred=mod.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.75      0.84      8504
           1       0.36      0.78      0.49      1496

    accuracy                           0.76     10000
   macro avg       0.65      0.77      0.67     10000
weighted avg       0.86      0.76      0.79     10000



In [6]:
#with SMOTENC sampling
model=LogisticRegression(max_iter=1000,random_state=42,solver='saga')
sampler=SMOTENC(random_state=42,categorical_features=[X.columns.get_loc(col) for col in cat_cols])
pipe=ImbPipeline(
    [
        ("preprocessor", preprocessor),
        ("sampler", sampler),   
        ("model", model)
    ]
)
mod=GridSearchCV(
    estimator=pipe,
    param_grid={
        'model__max_iter':[100,200,300,400,500,600,700,800,900,1000,2000],
        'model__solver':['saga','liblinear'],
        'model__class_weight':[{0:1,1:v}for v in np.linspace(1,20,10)]
    },
    cv=5,
    scoring='f1',
    n_jobs=2
)
mod.fit(X_test,y_test)
y_pred=mod.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.85      0.88      8504
           1       0.40      0.59      0.48      1496

    accuracy                           0.81     10000
   macro avg       0.66      0.72      0.68     10000
weighted avg       0.84      0.81      0.82     10000



In [7]:
#using undersampling and upweighting majority class 

from sklearn.utils import resample
 # Regular pipeline, no sampling

# Step 1: Manual downsampling
undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

# Step 2: Calculate upweight factor
original_majority_count = sum(y_train == 1)
new_majority_count = sum(y_resampled == 1)
upweight_factor = original_majority_count / new_majority_count

# Step 3: Create class weights
class_weight = {
    0: upweight_factor ,  # Upweight majority class
    1:20           # Keep minority class weight as 1
}

# Step 4: Use regular pipeline (no sampling step)
model = LogisticRegression(class_weight=class_weight, random_state=42)
pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model)
])

# Step 5: Fit on downsampled data
undersampled_model = RandomizedSearchCV(
    estimator=pipe,
    param_distributions={
        'model__max_iter': [100, 500, 1000, 2000],
        'model__solver': ['saga', 'liblinear']
    },
    n_iter=10,
    cv=5,
    scoring='f1',
    random_state=42
)

undersampled_model.fit(X_resampled, y_resampled)  # Use downsampled data
y_pred_undersampled = undersampled_model.predict(X_test)
print(classification_report(y_test, y_pred_undersampled))

C:\Users\nikhi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\nikhi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\nikhi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\nikhi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Pyth

              precision    recall  f1-score   support

           0       0.98      0.19      0.31      8504
           1       0.18      0.98      0.30      1496

    accuracy                           0.31     10000
   macro avg       0.58      0.58      0.31     10000
weighted avg       0.86      0.31      0.31     10000



In [8]:
pd.DataFrame(mod.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__class_weight,param_model__max_iter,param_model__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.418897,0.284883,0.029861,0.003857,"{0: 1, 1: 1.0}",100,saga,"{'model__class_weight': {0: 1, 1: 1.0}, 'model...",0.377465,0.416999,0.434540,0.395095,0.413889,0.407598,0.019587,1
1,2.839625,0.082097,0.034553,0.005122,"{0: 1, 1: 1.0}",100,liblinear,"{'model__class_weight': {0: 1, 1: 1.0}, 'model...",0.377465,0.416999,0.434540,0.395095,0.413889,0.407598,0.019587,1
2,3.144452,0.092828,0.033464,0.002673,"{0: 1, 1: 1.0}",200,saga,"{'model__class_weight': {0: 1, 1: 1.0}, 'model...",0.377465,0.416999,0.434540,0.395095,0.413889,0.407598,0.019587,1
3,2.888980,0.064034,0.036352,0.006791,"{0: 1, 1: 1.0}",200,liblinear,"{'model__class_weight': {0: 1, 1: 1.0}, 'model...",0.377465,0.416999,0.434540,0.395095,0.413889,0.407598,0.019587,1
4,3.155550,0.060184,0.032609,0.004254,"{0: 1, 1: 1.0}",300,saga,"{'model__class_weight': {0: 1, 1: 1.0}, 'model...",0.377465,0.416999,0.434540,0.395095,0.413889,0.407598,0.019587,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,5.604433,0.253330,0.030860,0.003746,"{0: 1, 1: 20.0}",300,saga,"{'model__class_weight': {0: 1, 1: 20.0}, 'mode...",0.304558,0.330808,0.307266,0.275455,0.268468,0.297311,0.022730,216
158,3.900651,0.077381,0.025422,0.001705,"{0: 1, 1: 15.777777777777779}",300,saga,"{'model__class_weight': {0: 1, 1: 15.777777777...",0.289918,0.300474,0.329726,0.299625,0.265722,0.297093,0.020584,217
210,8.631610,0.194589,0.030773,0.003377,"{0: 1, 1: 20.0}",700,saga,"{'model__class_weight': {0: 1, 1: 20.0}, 'mode...",0.300171,0.341207,0.281280,0.299327,0.263273,0.297052,0.025901,218
184,6.508912,0.669520,0.034107,0.008345,"{0: 1, 1: 17.88888888888889}",500,saga,"{'model__class_weight': {0: 1, 1: 17.888888888...",0.291405,0.277125,0.297674,0.292398,0.305869,0.292894,0.009406,219


In [9]:
pipe.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'),
                                    ['lead_bin', 'stay_bin', 'daypart',
                                     'sales_channel', 'trip_type', 'flight_day',
                                     'route', 'booking_origin',
                                     'wants_extra_baggage', 'wants_preferred_seat',
                                     'wants_in_flight_meals']),
                                   ('num', QuantileTransformer(),
                                    ['purchase_lead', 'length_of_stay',
                                     'flight_hour', 'flight_duration',
                                     'num_passengers'])])),
  ('model',
   LogisticRegression(class_weight={0: 1.0, 1: 20}, random_state=42))],
 'transform_input': None,
 'verbose': False,
 'preprocessor': ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'),
            

In [10]:
mod.best_params_

{'model__class_weight': {0: 1, 1: np.float64(1.0)},
 'model__max_iter': 100,
 'model__solver': 'saga'}

In [11]:
mod.best_estimator_

,steps,"[('preprocessor', ...), ('sampler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [12]:


y_proba = mod.predict_proba(X_test)[:,1]
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("F1:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

ROC-AUC: 0.8162906430317788
F1: 0.4762936873476023
Confusion Matrix:
 [[7188 1316]
 [ 617  879]]


## Stronger Model: Random Forest/XGBoost

In [13]:
pipe.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'),
                                    ['lead_bin', 'stay_bin', 'daypart',
                                     'sales_channel', 'trip_type', 'flight_day',
                                     'route', 'booking_origin',
                                     'wants_extra_baggage', 'wants_preferred_seat',
                                     'wants_in_flight_meals']),
                                   ('num', QuantileTransformer(),
                                    ['purchase_lead', 'length_of_stay',
                                     'flight_hour', 'flight_duration',
                                     'num_passengers'])])),
  ('model',
   LogisticRegression(class_weight={0: 1.0, 1: 20}, random_state=42))],
 'transform_input': None,
 'verbose': False,
 'preprocessor': ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'),
            

In [14]:

rf=RandomForestClassifier(
    random_state=42
)
rf_pipe=ImbPipeline( steps=
   [
       ("preprocessor",preprocessor),
       ("sampler",SMOTENC(categorical_features=[1,3,6,7,8,14,15,16])),
       ("model",rf)
   ]
)
param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10,  20],
    'model__min_samples_split': [2,5],
    'model__max_features': ['sqrt', 'log2'],
    'model__criterion': ['gini']
}

from sklearn.model_selection import RandomizedSearchCV

# More efficient alternative
mod = RandomizedSearchCV(
    estimator=rf_pipe,
    param_distributions=param_grid,  # Same param_grid
    n_iter=16,  
    cv=3,
    scoring='f1',
    n_jobs=-1,
    random_state=42
)
mod.fit(X_train,y_train)
y_pred=mod.predict(X_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.91      0.80      0.85      8504
           1       0.33      0.58      0.42      1496

    accuracy                           0.76     10000
   macro avg       0.62      0.69      0.64     10000
weighted avg       0.83      0.76      0.79     10000



In [15]:
# with undersampled majority class
rf=RandomForestClassifier(
    random_state=42
)
rf_pipe=ImbPipeline( steps=
   [
       ("preprocessor",preprocessor),
       ("sampler",SMOTENC(categorical_features=[1,3,6,7,8,14,15,16])),
       ("model",rf)
   ]
)
param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10,  20],
    'model__min_samples_split': [2,5],
    'model__max_features': ['sqrt', 'log2'],
    'model__criterion': ['gini']
}

from sklearn.model_selection import RandomizedSearchCV

# More efficient alternative
mod = RandomizedSearchCV(
    estimator=rf_pipe,
    param_distributions=param_grid,  # Same param_grid
    n_iter=16,  
    cv=3,
    scoring='f1',
    n_jobs=-1,
    random_state=42
)
mod.fit(X_resampled,y_resampled)
y_pred=mod.predict(X_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.93      0.74      0.82      8504
           1       0.31      0.66      0.42      1496

    accuracy                           0.73     10000
   macro avg       0.62      0.70      0.62     10000
weighted avg       0.83      0.73      0.76     10000



In [16]:

undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

model=RandomForestClassifier(
    class_weight={0:1,1:20},random_state=42
)
pipe=Pipeline( 
   [
       ("preprocessor",preprocessor),
       ("model",model)
   ]
)
coarse_grid={
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [10, 20],
    'model__max_features':['sqrt','log2'],
    'model__class_weight':[{0:1,1:x} for x in np.linspace(1,20,1)]
    
}
coarse_model=RandomizedSearchCV(
    estimator=pipe,
    param_distributions=coarse_grid,
    
    n_iter=10,
    cv=2,

    verbose=1,
    scoring='roc_auc',
    refit='f1_score',
    n_jobs=-1,
    random_state=42
)
coarse_model.fit(X_resampled,y_resampled)
y_pred=coarse_model.predict(X_test)
print(classification_report(y_test,y_pred))


Fitting 2 folds for each of 10 candidates, totalling 20 fits
              precision    recall  f1-score   support

           0       0.88      0.94      0.91      8504
           1       0.45      0.28      0.34      1496

    accuracy                           0.84     10000
   macro avg       0.66      0.61      0.63     10000
weighted avg       0.82      0.84      0.82     10000



In [17]:


# Sampler (you can switch between SMOTE or ROS)
sampler = SMOTE(random_state=42)

# Model
rf = RandomForestClassifier(
    random_state=42,
    class_weight="balanced_subsample"  )

# Pipeline
rf_pipe = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("sampler", sampler),   
    ("model", rf)
])

# Param grid for search
rf_params = {
    "model__n_estimators": [100, 200],
    "model__max_depth": [10, 20, None],
    "model__max_features": ["sqrt", "log2"]
}



rf_search = RandomizedSearchCV(
    estimator=rf_pipe,
    param_distributions=rf_params,
    n_iter=10,  
    cv=3,
    scoring="f1",  
    n_jobs=-1,
    random_state=42
)

rf_search.fit(X_train, y_train)


NameError: name 'SMOTE' is not defined

In [ ]:
pd.DataFrame(coarse_model.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_estimators,param_model__max_features,param_model__max_depth,param_model__class_weight,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
3,31.655366,0.044095,1.201481,0.000049,200,sqrt,20,"{0: 1, 1: 1.0}","{'model__n_estimators': 200, 'model__max_featu...",0.862857,0.871199,0.867028,0.004171,1
9,14.881245,0.135143,0.717639,0.000999,100,sqrt,20,"{0: 1, 1: 1.0}","{'model__n_estimators': 100, 'model__max_featu...",0.861684,0.869382,0.865533,0.003849,2
0,7.750992,0.238928,0.807643,0.008050,100,log2,20,"{0: 1, 1: 1.0}","{'model__n_estimators': 100, 'model__max_featu...",0.852905,0.854360,0.853632,0.000728,3
7,13.994172,0.306039,1.221684,0.013987,200,log2,20,"{0: 1, 1: 1.0}","{'model__n_estimators': 200, 'model__max_featu...",0.850870,0.853511,0.852190,0.001321,4
1,4.194893,0.045122,0.525506,0.006288,50,log2,20,"{0: 1, 1: 1.0}","{'model__n_estimators': 50, 'model__max_featur...",0.848167,0.845295,0.846731,0.001436,5
8,2.913774,0.080370,0.666214,0.035254,100,log2,10,"{0: 1, 1: 1.0}","{'model__n_estimators': 100, 'model__max_featu...",0.792132,0.795238,0.793685,0.001553,6
4,5.469851,0.171538,1.283536,0.074018,200,log2,10,"{0: 1, 1: 1.0}","{'model__n_estimators': 200, 'model__max_featu...",0.793824,0.793208,0.793516,0.000308,7
5,8.621422,0.307433,1.166748,0.003604,200,sqrt,10,"{0: 1, 1: 1.0}","{'model__n_estimators': 200, 'model__max_featu...",0.784981,0.788920,0.786950,0.001969,8
6,4.737646,0.033350,0.691757,0.063108,100,sqrt,10,"{0: 1, 1: 1.0}","{'model__n_estimators': 100, 'model__max_featu...",0.782515,0.788772,0.785643,0.003128,9
2,2.594317,0.232683,0.564181,0.029487,50,sqrt,10,"{0: 1, 1: 1.0}","{'model__n_estimators': 50, 'model__max_featur...",0.780823,0.786887,0.783855,0.003032,10


In [ ]:
mod.best_params_

{'model__n_estimators': 200,
 'model__min_samples_split': 5,
 'model__max_features': 'log2',
 'model__max_depth': 10,
 'model__criterion': 'gini'}

In [ ]:
mod.best_estimator_

,steps,"[('preprocessor', ...), ('sampler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
pd.DataFrame(mod.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_estimators,param_model__min_samples_split,param_model__max_features,param_model__max_depth,param_model__criterion,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
7,8.081139,0.375015,0.599749,0.105606,200,5,log2,10,gini,"{'model__n_estimators': 200, 'model__min_sampl...",0.414880,0.406871,0.401798,0.407850,0.005386,1
5,9.393741,0.275540,0.493131,0.015560,200,2,log2,10,gini,"{'model__n_estimators': 200, 'model__min_sampl...",0.409816,0.407368,0.402044,0.406409,0.003245,2
3,17.324061,0.546618,0.484480,0.017068,200,5,sqrt,10,gini,"{'model__n_estimators': 200, 'model__min_sampl...",0.414905,0.399860,0.400221,0.404995,0.007009,3
1,16.962746,0.313989,0.501728,0.002738,200,2,sqrt,10,gini,"{'model__n_estimators': 200, 'model__min_sampl...",0.412429,0.403367,0.397711,0.404503,0.006062,4
14,11.739645,0.057632,0.387739,0.016070,100,5,log2,20,gini,"{'model__n_estimators': 100, 'model__min_sampl...",0.408124,0.405234,0.398330,0.403896,0.004109,5
15,15.774118,0.807962,0.503159,0.058811,200,5,log2,20,gini,"{'model__n_estimators': 200, 'model__min_sampl...",0.405318,0.403838,0.402400,0.403852,0.001191,6
4,5.372017,0.140344,0.298893,0.020460,100,2,log2,10,gini,"{'model__n_estimators': 100, 'model__min_sampl...",0.400068,0.408948,0.401927,0.403648,0.003824,7
12,12.961930,0.720540,0.345928,0.015849,100,2,log2,20,gini,"{'model__n_estimators': 100, 'model__min_sampl...",0.407729,0.399198,0.403119,0.403349,0.003487,8
6,4.963914,0.053931,0.278250,0.012054,100,5,log2,10,gini,"{'model__n_estimators': 100, 'model__min_sampl...",0.402985,0.403679,0.402332,0.402999,0.000550,9
2,9.088432,0.572576,0.357001,0.070977,100,5,sqrt,10,gini,"{'model__n_estimators': 100, 'model__min_sampl...",0.406097,0.400070,0.401895,0.402687,0.002523,10


,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__criterion,param_model__max_depth,param_model__max_features,param_model__min_samples_split,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
18,0,1250,0.667906,0.064930,0.047168,0.006058,gini,20,log2,10,...,0.8592,0.017417,1,0.8370,0.8350,0.857,0.8640,0.8560,0.8498,0.011617
19,0,1250,1.154541,0.074112,0.068070,0.012918,gini,20,log2,10,...,0.8592,0.017417,1,0.8370,0.8340,0.855,0.8640,0.8540,0.8488,0.011444
43,0,1250,1.132383,0.053901,0.062557,0.003049,entropy,20,log2,10,...,0.8592,0.017417,1,0.8370,0.8350,0.855,0.8640,0.8550,0.8492,0.011285
42,0,1250,0.590408,0.035077,0.045740,0.003126,entropy,20,log2,10,...,0.8592,0.017417,1,0.8380,0.8350,0.856,0.8660,0.8560,0.8502,0.011805
17,0,1250,1.438332,0.070335,0.067300,0.007412,gini,20,log2,2,...,0.8584,0.017817,5,0.8770,0.8610,0.889,0.9060,0.8960,0.8858,0.015587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,1,2500,8.730350,0.432294,0.067252,0.017294,entropy,None,None,2,...,0.8420,0.018719,90,1.0000,1.0000,1.000,1.0000,1.0000,1.0000,0.000000
60,1,2500,13.734163,0.353122,0.079998,0.013221,entropy,20,None,10,...,0.8416,0.018304,92,0.9375,0.9255,0.915,0.9205,0.9245,0.9246,0.007432
11,0,1250,8.884584,0.898478,0.064576,0.009728,gini,None,None,10,...,0.8384,0.026605,93,0.9330,0.9400,0.935,0.9530,0.9460,0.9414,0.007338
9,0,1250,9.528269,0.625291,0.058470,0.004707,gini,None,None,2,...,0.8344,0.027317,94,1.0000,1.0000,1.000,1.0000,1.0000,1.0000,0.000000


## Threshold Tuning for F1

## Feature Importance (LR + Tree)

## Top 5 Drivers + 1-line interpretations

## Business Actions

## Model Card (short)